In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
from torch.utils.data import Dataset, DataLoader
from time import perf_counter 
import math
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
%matplotlib inline
 
def to_torch(state):
    state = torch.from_numpy(state).float()
    return state
 
class RNN(nn.Module):
    def __init__(self, p, q, output_size):
        super(RNN, self).__init__()
        self.p = p
        self.q = q

        self.hidden_dim = q
        self.feature_size = p
        self.output_size = output_size
        self.rnn = nn.RNN(self.feature_size, self.hidden_dim)
        self.hidden2output = nn.Linear(self.hidden_dim, output_size)
 
    def forward(self, input_y, h = None):  
        samples = input_y
        rnn_out, last_rnn_hidden = self.rnn(samples, h)
        output = self.hidden2output(rnn_out.view(-1, self.hidden_dim))
        return output.view(samples.shape[0],samples.shape[1],self.output_size), last_rnn_hidden

In [2]:
### " F-norm of A
class L2LossFun(nn.Module):
    
    def __init__(self):
        super(L2LossFun, self).__init__()
    def forward(self, A_Est, A_True):
        gap = math.sqrt(np.sum((A_Est - A_True)**2))
        return gap

class LinfLossFun(nn.Module):
    
    def __init__(self):
        super(LinfLossFun, self).__init__()
    def forward(self, A_Est, A_True):
        gap = np.max(np.abs(A_Est-A_True))
        return gap

In [3]:
dfP = pd.read_csv("co2_327_45.csv") 
df = np.array(dfP)
### "Data preprocessing
scaler = MinMaxScaler(feature_range=(0, 1))
df = scaler.fit_transform(df)

In [4]:
# Hyper Parameters
N = 45
output_size = 45
Smp_size = 100
q = 1 #hidden dimension
lag = 20
Pred_size = len(df)-Smp_size - lag
# the layer is set to be 1 and fixed

In [5]:
names = {}
names["predErrorRNN1"] =  []
names["predErrorRNN1Linf"] =  []
names["LTRRNN1_pred"] = []

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn.parameter import Parameter
import copy
from copy import deepcopy
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pickle
import matplotlib.pyplot as plt
%matplotlib inline
 
# initialization
p , q, output_size, K = (N, 1, N, 100)

# transform the time series involving
def create_dataset(y, look_back=1):
    dataYp,dataYc = [], [] #Ypast, Ycurrent
    for i in range(len(y)-look_back):
        dataYp.append(y[i,:])
        dataYc.append(y[i+look_back, :])
    return np.array(dataYp), np.array(dataYc)
 
# transform data to tensor in torch
def to_torch(state):
    state = torch.from_numpy(state).float()
    return state

In [7]:
distance = L2LossFun()
distanceLinf = LinfLossFun()

t1_start = perf_counter() 

for k in range(pred_size):
    y = df[(Smp_size+k+lag-100):(Smp_size+k+lag+1),:] #Fixed size at 100!Need to readjust
    inY, outY = create_dataset(y=y, look_back=1) 
    # split into train and test sets
    train_size = 100                                  #Fixed size at 100!Need to readjust
    test_size = 1
    trainInY,trainOutY = inY[0:train_size-1], outY[0:train_size-1]
    testInY,testOutY =  inY[train_size-1:], outY[train_size-1:]
    trainInY = np.reshape(trainInY, (trainInY.shape[0], 1, p))
    trainOutY = np.reshape(trainOutY, (trainOutY.shape[0], 1, p))
    testInY = np.reshape(testInY, (testInY.shape[0], 1, p))
    testOutY = np.reshape(testOutY, (testOutY.shape[0], 1, p))

    model = RNN(p = N,q = 1, output_size = output_size)
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr = 0.01, momentum=0.9)

    loss_last = 1000
    loss_new = 0

    i = 0
    while abs(loss_last - loss_new)  > 0.000001:
        if i > 0:
            loss_last = loss_new  

        model.zero_grad()
        target = torch.from_numpy(trainOutY).float()
        output,_ = model(torch.from_numpy(trainInY).float())

        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        output, h = model(torch.from_numpy(trainInY).float())
        loss_new = loss.item()
        i = i + 1
        #print(loss_new)
    testPredict,_ = model(torch.from_numpy(testInY).float(), h)
    testPredict = testPredict.detach().numpy()
    # invert predictions
    #testPredict_r = scaler.inverse_transform(testPredict[:,0, :])
    #testY_r = scaler.inverse_transform(testOutY[:,0,:])
    testPredict = testPredict[:,0, :]
    testOutY = testOutY[:,0,:]
    names["LTRRNN1_pred"].append(testPredict)
    predError = distance(testPredict, testOutY)
    names["predErrorRNN1"].append(predError)
    print("PredError is {}.".format(predError))
    predErrorLinf = distanceLinf(testPredict, testOutY)
    names["predErrorRNN1Linf"].append(predErrorLinf)
    print("PredErrorLinf is {}.".format(predErrorLinf))
    
    print(k)

t1_stop = perf_counter() 
print("Elapsed time during the whole program in seconds:", 
                                        t1_stop-t1_start) 
Real_100RNN2s = names
torch.save(Real_100RNN2s, "Real_100RNN2s.py")

PredError is 0.9794152196154056.
PredErrorLinf is 0.42178117226011674.
0
PredError is 1.1175721860783059.
PredErrorLinf is 0.4568493962287903.
1
PredError is 0.4728309958496498.
PredErrorLinf is 0.14047815310595413.
2
PredError is 0.6601884671408236.
PredErrorLinf is 0.2808056166010946.
3
PredError is 0.6454431732453318.
PredErrorLinf is 0.4436774023705148.
4
PredError is 1.072909057550083.
PredErrorLinf is 0.40746612423980455.
5
PredError is 0.84463068502203.
PredErrorLinf is 0.4420682191848755.
6
PredError is 0.5783005860383496.
PredErrorLinf is 0.21753781916200243.
7
PredError is 0.6436104292916567.
PredErrorLinf is 0.4918628931045532.
8
PredError is 0.5899546531277108.
PredErrorLinf is 0.25800762969308644.
9
Elapsed time during the whole program in seconds: 72.67609129599805
